# Complete chunk analysis
---
This notebook is where we test a comprehensive chunk analysis, all in one go (i.e. one LLM run). While it sounds ambitious, there's a chance that a well defined prompt paired with a good enough model (reasoning LLM?) could do:
- Policy proposal extraction;
- Topic segmentation;
- Sentiment analysis;
- Hate speech detection;
- Political compass estimation;

all at the same time.

If this doesn't perform well enough, we can always break it down into individual steps and run them separately. But let's give it a try first.

## Setup

### Import libraries

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Literal
from openai import OpenAI

In [ ]:
from polids.config import settings
from polids.structured_analysis.openai import OpenAIStructuredChunkAnalyzer

### Set parameters

In [ ]:
system_prompt = "You are an expert political text analyst with deep knowledge of political ideologies, policy frameworks, and manifesto analysis. Your task is to analyze segments from political party electoral manifestos, focusing on precision, accuracy, and strict adherence to the provided schema."


def user_prompt_format(text):
    return f"""**Analysis process**:
1. Policy Proposals:
- Carefully read the text to identify *concrete and specific* policy proposals.
- A valid policy proposal MUST:
  * Be a specific, actionable governmental commitment
  * Contain a clear, measurable action or initiative
  * Be concise (single short sentence)
- Do NOT include:
  * Vague ideological statements
  * General party values or principles without specific actions
  * Rhetorical flourishes or campaign slogans
  * Background information or contextual details
- Summarize each distinct proposal concisely, focusing *only* on the core action. Avoid including surrounding justification or filler text from the source. Aim for quality over quantity – extract only truly specific actions.
- If the text contains NO specific policy proposals, provide an empty list.
- Translate any non-English proposals into English.
- Provide the proposals as a list under field name: policy_proposals.

2. Sentiment:
- Evaluate only the *emotional tone* (not content or ideology).
- Select exactly ONE option: "positive" (optimistic, encouraging), "negative" (critical, pessimistic), or "neutral" (balanced, matter-of-fact).
- Provide this value under field name: sentiment.

3. Topic:
- Identify the *single, most dominant* political topic discussed in the chunk.
- Choose ONE specific keyword in English (avoid combining multiple topics).
- Common topics include but are NOT limited to: "economy", "healthcare", "education", "migration", "security", "environment", "foreign policy", "culture", "democracy", "justice".
- DO NOT use compound topics - select the most central focus only.
- Provide the chosen keyword under field name: topic.

4. Hate Speech Analysis:
- Analyze the text for hate speech based on the provided definition (hostility/prejudice/discrimination against groups based on characteristics).
- For field hate_speech:
  a. Set to true ONLY if the text contains explicit hostility, encourages discrimination, or promotes prejudice against identifiable groups.
  b. For subfield reason:
    - If hate speech is present: Quote SPECIFIC phrases and explain how they constitute hate speech.
    - If no hate speech: Briefly explain why the content was determined safe.
  c. For subfield targeted_groups: List ONLY groups explicitly targeted (if any) or leave empty.
- If hate_speech is false, targeted_groups should be empty.
- Place values under field name: hate_speech.

5. Political Compass Analysis:
- Assess political orientation with particular attention to explicit policy indicators:
  a. Economic:
    - "left": Strong indicators include support for wealth redistribution, expanded public services, market regulation, nationalization.
    - "right": Strong indicators include tax reduction, privatization, deregulation, free market emphasis, cutting government spending.
    - "center": Represents a mixed approach, balancing market principles with social welfare or targeted regulation (e.g., social market economy).
  b. Social:
    - "libertarian": Emphasizes personal freedoms, minimal state intervention in private life, civil liberties expansion.
    - "authoritarian": Emphasizes order, discipline, social conformity, expanded state powers over individuals.
    - "center": Balanced approach to personal freedoms and societal order.
- Provide selections under field name: political_compass with subfields economic and social.

**Task**:
Analyze the Markdown formatted text, applying the process described above.

**Input text**:
```markdown
{text}
```"""

## Load chunks to parse through

We're going to start from pre-chunked list of strings, so as to avoid dependencies on previous steps of the pipeline.

In [ ]:
chunks_per_type = {
    "pnr_hate_speech": """- Recuperar a identidade ocidental e a matriz cultural cristã;
- Repatriar imigrantes subsídio-dependentes, criminosos, ilegais e que sejam inadaptáveis em termos de cultura, costumes e comportamento;
- Cortar os apoios e subsídios (discriminação positiva) para as minorias étnicas;
- Travar o crescimento do Islão em Portugal e proibir a construção de novas mesquitas;
- Anular a lei do “casamento” entre pessoas do mesmo sexo;
- Limitar o acesso às forças armadas e demais forças de segurança só aos portugueses de raiz.""",
    "livre_sustainability": "**Criar uma taxa universal sobre o carbono**, no quadro de uma reforma fiscal ambiental, internalizando dessa forma as externalidades geradas, assegurando equidade social através de uma abordagem que resulte em neutralidade fiscal, por exemplo através da redução da tributação sobre o trabalho, complementando com a eliminação de subsídios ou ecotaxas ambientalmente prejudiciais, aplicando os princípios do poluidor-pagador e utilizador-pagador e incentivando o pagamento de serviços dos ecossistemas ou o investimento em eficiência energética ou demais medidas de caráter ambiental.",
    "il_economy": """### SIMPLIFICAÇÃO E DESAGRAVAMENTO DO IRS COM INTRODUÇÃO DE TAXA ÚNICA DE 15%

- Implementação de uma taxa única de IRS de 15%, aplicada por igual a todos os rendimentos e para todos os contribuintes

- Isenção de IRS para rendimentos de trabalho até remuneração mensal de cerca de €664

- Isenção adicional de 200€ mensais por filho dependente e por progenitor (400€ em caso de famílias monoparentais)

- Eliminação de todas as deduções e benefícios fiscais em sede de IRS, com exceção das mencionadas no ponto anterior

- Transitoriamente, um sistema de duas taxas: 15% para rendimentos até 30.000€ e 28% no remanescente""",
    "volt_ideology": """# Identidade e Visão do Volt

Imagina-te numa situação onde tu e os teus concidadãos teriam de desenhar de raiz a sociedade em que viveriam. Que tipo de sociedade escolherias?

O filósofo John Rawls propôs que, antes de escolher, cada um de nós colocasse um ‘véu da ignorância’. Ao colocar este véu, desconheceríamos o nosso ponto de partida ou as nossas circunstâncias na sociedade, ou seja, não saberíamos qual o nosso nível de riqueza, o nosso estatuto, a nossa orientação sexual, não saberíamos nada sobre o nosso lugar no mundo. Não possuindo estas informações, e procurando o nosso próprio bem-estar, seria certamente consensual a construção de uma sociedade em que ter uma vida digna é um direito real e universal, onde a liberdade de qualquer pessoa é apenas limitada pelo direito à liberdade do nosso próximo e onde todos têm o mesmo nível de oportunidades para prosperar e alcançar a sua própria definição de felicidade.

Esta é a **visão** do Volt de uma sociedade justa, de um Portugal e de uma Europa para todos, e esta funciona como ponto de partida para a ideologia que guia a nossa ação política.

Somos **progressistas** porque não nos resignamos ao estado atual das coisas, opondo-nos ao conservadorismo e procurando promover soluções inovadoras para os nossos problemas comuns e para o aperfeiçoamento da condição humana em sociedade, soluções onde prevaleçam os valores da liberdade, da igualdade de oportunidades, da solidariedade, da dignidade humana e dos direitos humanos. Além de social, o progressismo que defendemos é um que inclui o **ecologismo** porque vemos o bem-estar do planeta e dos outros seres vivos como um fim em si mesmo e como uma condição necessária para o bem-estar da sociedade. Por isto mesmo, acreditamos que a necessidade de sustentabilidade impõe limitações importantes à forma como a sociedade é conduzida e em todas as soluções que defendemos temos a preocupação de encontrar uma harmonia com o que nos rodeia e com o que nos sustenta vendo como prioridade a preservação do meio ambiente, a utilização responsável de recursos minerais e energéticos e o combate às alterações climáticas.

Somos **pragmáticos** porque procuramos ser coerentes com a nossa visão e valores propondo e defendendo o que de facto funcione com base no consenso, no uso da razão e da evidência científica e nunca em dogmatismos ideológicos ou no egoísmo na procura do poder. Colocamo-nos ao centro do espetro político na forma como vemos o papel do Estado, pois vemos este como um meio e não como um fim, acreditamos que o Estado deve intervir tão pouco e tão rápido quanto possível e tanto e por quanto tempo for necessário.

Somos **europeístas** porque, sendo Portugueses, sentimo-nos também Europeus. Vemos a União Europeia como parte da nossa identidade e futuro e, reconhecendo a interdependência do mundo em que vivemos, olhamos para o aprofundamento do projeto europeu e para a construção de uma federação europeia como o caminho para a resolução dos grandes desafios que enfrentamos. Queremos uma Europa unida e mais democrática que valoriza os seus cidadãos e os seus residentes, que não deixa ninguém para trás e que cria e mantém as condições necessárias para a realização do potencial único de cada um e da comunidade no seu todo. Uma Europa que se esforça continuamente para alcançar, em conjunto e de forma solidária, os mais elevados padrões de desenvolvimento humano, social, ambiental e técnico e que possa ajudar a guiar o resto do mundo através do exemplo na prossecução desses mesmos padrões, sendo um promotor chave da paz, da justiça, da prosperidade e da sustentabilidade global.

Em 2017, como resposta aos movimentos nacionalistas e populistas, e em específico como resposta ao Brexit, um conjunto de jovens de várias nacionalidades criou este movimento político a que foi dado o nome de Volt. Foi escolhido este nome porque este simboliza, da mesma forma por toda a Europa, aquilo que queremos fazer, dar pelas nossas próprias mãos uma nova energia à política.

O Volt é o meio pelo qual podemos ter uma voz na política e pelo qual podemos empoderar outros para também a terem, contribuindo assim para uma sociedade cada vez mais democrática e justa. Queremos fazer a política de uma forma diferente, uma política com uma liderança forte e confiante mas ao mesmo tempo partilhada de forma a evitar a concentração de poder e o culto de personalidade, uma política onde as pessoas são ouvidas e onde a forma pela qual alcançamos a mudança é começando por nós mesmos de forma a dar o exemplo aos que nos rodeiam. Acima de tudo, reconhecemos a necessidade de não deixar a política nas mãos de outros. Queremos assim assumir a responsabilidade individual pela construção de um futuro melhor para todos acreditando que com solidariedade não há nenhuma atitude, grande ou pequena, que seja insignificante pois é através da repetição de muitas destas que é construída uma solidariedade de facto na sociedade.

«A Europa não se fará de uma só vez, nem de acordo com um plano único. Far-se-á através de realizações concretas que criarão, antes de mais, uma solidariedade de facto.»

Robert Schuman, um dos fundadores do que é hoje a União Europeia, 1950""",
}

## Test structured output analysis

### Initialize the LLM client

In [ ]:
client = OpenAI(api_key=settings.openai_api_key)

### Define the output structure

In [ ]:
class HateSpeechDetection(BaseModel):
    """
    Model representing the analysis of hate speech within a text.
    """

    hate_speech: bool = Field(
        description=(
            "Boolean indicating if the text contains hate speech. Hate speech refers to any public expression "
            "that communicates hostility, animosity, or encourages violence, prejudice, discrimination, or "
            "intimidation against individuals or groups based on identifiable characteristics "
            "(e.g. race, ethnicity, national origin, religion, gender identity, sexual orientation, disability, age)."
        ),
    )
    reason: str = Field(
        description=(
            "Detailed explanation justifying the classification as hate speech or not. "
            "Include specific parts of the text that lead to this decision. If no hate speech is detected, explain why the content was determined safe."
        ),
    )
    targeted_groups: List[str] = Field(
        default_factory=list,
        description=(
            "List of groups or protected characteristics that are explicitly targeted by hate speech in the text. "
            "Examples include: 'race', 'religion', 'sexual orientation', 'gender identity', 'disability'. "
            "If no group is targeted, this list should be empty."
        ),
    )


class PoliticalCompass(BaseModel):
    """
    Model for the political compass analysis, representing two primary dimensions:
    - Economic: perspective on economic organization (left vs. right)
    - Social: perspective on personal freedom and state intervention (libertarian vs. authoritarian)
    """

    economic: Literal["left", "center", "right"] = Field(
        description=(
            "Economic stance on the political spectrum. 'left' suggests support for cooperative or state-driven "
            "economic models, 'right' indicates a free-market approach emphasizing individual competition, and "
            "'center' represents a moderate position combining elements of both views."
        ),
    )
    social: Literal["libertarian", "center", "authoritarian"] = Field(
        description=(
            "Social stance on the political spectrum. 'libertarian' represents maximal personal freedom with minimal state control, "
            "'authoritarian' indicates a preference for obedience to authority and strict social order, and "
            "'center' denotes a balanced or moderate view."
        ),
    )


class ManifestoChunkAnalysis(BaseModel):
    """
    Model for a comprehensive analysis of a segment (chunk) of a political party's electoral manifesto.
    Includes:
    - Policy proposals extraction
    - Sentiment analysis
    - Dominant political topic identification
    - Hate speech detection
    - Political compass positioning
    """

    policy_proposals: List[str] = Field(
        description=(
            "A list of one or more policy proposals extracted from the manifesto chunk. Each entry should be a concise, "
            "specific statement describing a proposed action to address a political issue. Proposals should be translated to English if needed."
        ),
    )
    sentiment: Literal["positive", "negative", "neutral"] = Field(
        description=(
            "The overall sentiment of the manifesto chunk. Valid values include 'positive', 'negative', or 'neutral', "
            "reflecting the emotional tone or attitude conveyed in the text."
        ),
    )
    topic: str = Field(
        description=(
            "The dominant political topic of the manifesto chunk. This should be captured in one or two keywords in English. "
            "Examples: 'economy', 'healthcare', 'education', 'migration', 'transport', 'science', 'sustainability', "
            "'welfare', 'social causes', 'ideology', 'infrastructure', 'business', 'technology', 'urban design'."
        ),
    )
    hate_speech: HateSpeechDetection = Field(
        description="Nested analysis of hate speech detection within the manifesto chunk.",
    )
    political_compass: PoliticalCompass = Field(
        description="Nested analysis mapping the manifesto chunk to positions on the political compass.",
    )

### GPT 4o mini

In [ ]:
chunk_analysis = {example_name: None for example_name in chunks_per_type.keys()}
for example_name, example_chunk in chunks_per_type.items():
    completion = client.beta.chat.completions.parse(
        # Using the mini version for cheaper processing; setting a specific version for reproducibility
        model="gpt-4o-mini-2024-07-18",
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt_format(example_chunk),
            },
        ],
        response_format=ManifestoChunkAnalysis,  # Specify the schema for the structured output
        temperature=0,  # Low temperature should lead to less hallucination
        seed=42,  # Fix the seed for reproducibility
    )
    chunk_analysis[example_name] = completion.choices[0].message.parsed
    assert isinstance(chunk_analysis[example_name], ManifestoChunkAnalysis), (
        "Output does not match the expected schema."
    )
chunk_analysis

In [ ]:
chunk_analysis["pnr_hate_speech"].model_dump()

In [ ]:
chunk_analysis["livre_sustainability"].model_dump()

In [ ]:
chunk_analysis["il_economy"].model_dump()

GPT 4o mini has gotten the economic stance of IL wrong, as it should be right-wing.

In [ ]:
chunk_analysis["volt_ideology"].model_dump()

### GPT 4o

In [ ]:
chunk_analysis = {example_name: None for example_name in chunks_per_type.keys()}
for example_name, example_chunk in chunks_per_type.items():
    completion = client.beta.chat.completions.parse(
        # Using the larger GPT 4o model to ensure better output quality; setting a specific version for reproducibility
        model="gpt-4o-2024-11-20",
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt_format(example_chunk),
            },
        ],
        response_format=ManifestoChunkAnalysis,  # Specify the schema for the structured output
        temperature=0,  # Low temperature should lead to less hallucination
        seed=42,  # Fix the seed for reproducibility
    )
    chunk_analysis[example_name] = completion.choices[0].message.parsed
    assert isinstance(chunk_analysis[example_name], ManifestoChunkAnalysis), (
        "Output does not match the expected schema."
    )
chunk_analysis

In [ ]:
chunk_analysis["pnr_hate_speech"].model_dump()

In [ ]:
chunk_analysis["livre_sustainability"].model_dump()

In [ ]:
chunk_analysis["il_economy"].model_dump()

In [ ]:
chunk_analysis["volt_ideology"].model_dump()

GPT 4o looks better than the mini version. It also wasn't any slower than mini, just more expensive.

### o3 mini

In [ ]:
chunk_analysis = {example_name: None for example_name in chunks_per_type.keys()}
for example_name, example_chunk in chunks_per_type.items():
    completion = client.beta.chat.completions.parse(
        # Setting a specific version for reproducibility
        model="o3-mini-2025-01-31",
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt_format(example_chunk),
            },
        ],
        response_format=ManifestoChunkAnalysis,  # Specify the schema for the structured output
        seed=42,  # Fix the seed for reproducibility
    )
    chunk_analysis[example_name] = completion.choices[0].message.parsed
    assert isinstance(chunk_analysis[example_name], ManifestoChunkAnalysis), (
        "Output does not match the expected schema."
    )
chunk_analysis

In [ ]:
chunk_analysis["pnr_hate_speech"].model_dump()

In [ ]:
chunk_analysis["livre_sustainability"].model_dump()

In [ ]:
chunk_analysis["il_economy"].model_dump()

In [ ]:
chunk_analysis["volt_ideology"].model_dump()

o3 mini takes at least twice as long as GPT 4o and it doesn't look significantly better than GPT 4o. However, it could potentially be cheaper than GPT 4o and leverage its reasoning capabilities for more complex input text.

### Implemented solution

In [ ]:
structured_analyzer = OpenAIStructuredChunkAnalyzer()
outputs = {
    example_name: structured_analyzer.process(example_text)
    for example_name, example_text in chunks_per_type.items()
}
outputs